테이블 확인

In [2]:
import pymysql
import pandas as pd
import os
import json

In [4]:
connection = pymysql.connect(
    host='15.165.195.216',
    port=3306,
    user='root',
    password='team266',
    database='team2',
    charset='utf8'
)
connection

try:
    with connection.cursor() as cursor:
        cursor.execute("SHOW TABLES;")
        result = cursor.fetchall()
        
        if result:
            print('존재 목록:')
            for table in result:
                print(table[0])
        else:
            print('미존재')
            
except Exception as e:
    print(f"테이블 조회 중 오류 발생: {e}")

finally:
    connection.close()

존재 목록:
tb_address
tb_parking
tb_traffic


테이블 생성

In [13]:
connection = pymysql.connect(
    host='15.165.195.216',
    port=3306,
    user='root',
    password='team266',
    database='team2',
    charset='utf8'
)

create_table_query = """
CREATE TABLE IF NOT EXISTS tb_traffic (
    traffic_id INTEGER NOT NULL AUTO_INCREMENT,
    roadName VARCHAR(100) NULL,
    roadDrcType FLOAT NULL,
    linkNo FLOAT NULL,
    linkId BIGINT NULL,
    startNodeId BIGINT NULL,
    endNodeId BIGINT NULL,
    speed FLOAT NULL,
    travelTime FLOAT NULL,
    createdDate BIGINT NULL,
    PRIMARY KEY (traffic_id)
);
"""

try:
    with connection.cursor() as cursor:
        cursor.execute("DROP TABLE IF EXISTS tb_traffic;")
        cursor.execute(create_table_query)
        connection.commit()
        print('테이블 생성 완료')
except Exception as e:
   print(f"오류{e}")
finally:
    connection.close() 

테이블 생성 완료


데이터 파일 확인

In [11]:
directory_path = r'C:\Users\ran14\Workspace\park-to-play Workspace\db 코드\its_traffic'

# 경로의 모든 csv파일 확인
csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]

print(f"총 {len(csv_files)}개의 파일")

# 첫 번째 파일 확인
sample_file = os.path.join(directory_path, csv_files[0])
print(sample_file)
df = pd.read_csv(sample_file, encoding='utf-8')
print(df.info())
print(df.head())

총 286개의 파일
C:\Users\ran14\Workspace\park-to-play Workspace\db 코드\its_traffic\its_traffic_data_2024-10-01_184218.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44772 entries, 0 to 44771
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   roadName     44772 non-null  object 
 1   roadDrcType  0 non-null      float64
 2   linkNo       0 non-null      float64
 3   linkId       44772 non-null  int64  
 4   startNodeId  44772 non-null  int64  
 5   endNodeId    44772 non-null  int64  
 6   speed        44772 non-null  float64
 7   travelTime   44772 non-null  float64
 8   createdDate  44772 non-null  int64  
dtypes: float64(4), int64(4), object(1)
memory usage: 3.1+ MB
None
  roadName  roadDrcType  linkNo      linkId  startNodeId   endNodeId  speed  \
0       종로          NaN     NaN  1000000301   1000008600  1000008900   11.5   
1       종로          NaN     NaN  1000000302   1000008900  1000007700   10.0   
2       종로

데이터 삽입

In [12]:
import csv

In [15]:
connection = pymysql.connect(
    host='15.165.195.216',
    port=3306,
    user='root',
    password='team266',
    database='team2',
    charset='utf8'
)

directory_path = r'C:\Users\ran14\Workspace\park-to-play Workspace\db 코드\its_traffic'

csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]

insert_query = """
INSERT INTO tb_traffic (roadName, roadDrcType, linkNo, linkId, startNodeId, endNodeId, speed, travelTime, createdDate)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

try:
    with connection.cursor() as cursor:
        # 각 CSV 파일을 하나씩 읽어 데이터베이스에 삽입
        for csv_file in csv_files:
            file_path = os.path.join(directory_path, csv_file)  # 각 파일의 전체 경로
            print(f"파일 로드 중: {file_path}")

            # CSV 파일 읽기
            with open(file_path, mode='r', encoding='utf-8') as f:
                csv_reader = csv.reader(f)
                header = next(csv_reader)  # 첫 번째 행(헤더) 건너뛰기

                # 각 행을 데이터베이스에 삽입
                for row in csv_reader:
                    cursor.execute(
                        insert_query,
                        (
                            row[0],
                            float(row[1]) if row[1] else None,
                            float(row[2]) if row[2] else None,
                            int(row[3]) if row[3] else None,
                            int(row[4]) if row[4] else None,
                            int(row[5]) if row[5] else None,
                            float(row[6]),
                            float(row[7]),
                            int(row[8]) if row[8] else None 
                        )
                    )
            # 각 파일별로 커밋
            connection.commit()
            print(f"'{csv_file}'삽입 완료")

except Exception as e:
    print(f"오류{e}")

finally:
    # 데이터베이스 연결 종료
    connection.close()
            

파일 로드 중: C:\Users\ran14\Workspace\park-to-play Workspace\db 코드\its_traffic\its_traffic_data_2024-10-01_184218.csv


KeyboardInterrupt: 

- cursor.execute() 대신 excutemany() 사용


In [ ]:
import pymysql
import csv
import os

In [16]:
connection = pymysql.connect(
    host='15.165.195.216',
    port=3306,
    user='root',
    password='team266',
    database='team2',
    charset='utf8'
)

directory_path = r'C:\Users\ran14\Workspace\park-to-play Workspace\db 코드\its_traffic'

csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]

insert_query = """
INSERT INTO tb_traffic (roadName, roadDrcType, linkNo, linkId, startNodeId, endNodeId, speed, travelTime, createdDate)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

try:
    with connection.cursor() as cursor:
        for csv_file in csv_files:
            file_path = os.path.join(directory_path, csv_file)
            print(f"파일 로드 중: {file_path}")

            # CSV 파일 읽기
            with open(file_path, mode='r', encoding='utf-8') as f:
                csv_reader = csv.reader(f)
                header = next(csv_reader)  # 헤더 건너뛰기

                # 행들을 리스트에 저장하여 한 번에 삽입
                batch_data = []
                for row in csv_reader:
                    batch_data.append((
                        row[0],  # roadName
                        float(row[1]) if row[1] else None,  # roadDrcType (비어있으면 None으로 처리)
                        float(row[2]) if row[2] else None,  # linkNo (비어있으면 None으로 처리)
                        int(row[3]) if row[3] else None,  # linkId (비어있으면 None으로 처리)
                        int(row[4]) if row[4] else None,  # startNodeId (비어있으면 None으로 처리)
                        int(row[5]) if row[5] else None,  # endNodeId (비어있으면 None으로 처리)
                        float(row[6]),  # speed
                        float(row[7]),  # travelTime
                        int(row[8]) if row[8] else None  # createdDate (비어있으면 None으로 처리)
                    ))

                # `executemany()`로 배치 삽입
                cursor.executemany(insert_query, batch_data)
                connection.commit()
                print(f"'{csv_file}'의 데이터가 성공적으로 삽입되었습니다.")

except Exception as e:
    print(f"오류{e}")

finally:
    connection.close()

파일 로드 중: C:\Users\ran14\Workspace\park-to-play Workspace\db 코드\its_traffic\its_traffic_data_2024-10-01_184218.csv
'its_traffic_data_2024-10-01_184218.csv'의 데이터가 성공적으로 삽입되었습니다.
파일 로드 중: C:\Users\ran14\Workspace\park-to-play Workspace\db 코드\its_traffic\its_traffic_data_2024-10-01_190219.csv
'its_traffic_data_2024-10-01_190219.csv'의 데이터가 성공적으로 삽입되었습니다.
파일 로드 중: C:\Users\ran14\Workspace\park-to-play Workspace\db 코드\its_traffic\its_traffic_data_2024-10-01_190607.csv
'its_traffic_data_2024-10-01_190607.csv'의 데이터가 성공적으로 삽입되었습니다.
파일 로드 중: C:\Users\ran14\Workspace\park-to-play Workspace\db 코드\its_traffic\its_traffic_data_2024-10-01_192612.csv
'its_traffic_data_2024-10-01_192612.csv'의 데이터가 성공적으로 삽입되었습니다.
파일 로드 중: C:\Users\ran14\Workspace\park-to-play Workspace\db 코드\its_traffic\its_traffic_data_2024-10-01_194609.csv
'its_traffic_data_2024-10-01_194609.csv'의 데이터가 성공적으로 삽입되었습니다.
파일 로드 중: C:\Users\ran14\Workspace\park-to-play Workspace\db 코드\its_traffic\its_traffic_data_2024-10-01_200609.csv
'its_t

판다스로 불러와 확인

In [17]:
connection = pymysql.connect(
    host='15.165.195.216',
    port=3306,
    user='root',
    password='team266',
    database='team2',
    charset='utf8'
)

query = "SELECT * FROM tb_traffic;"
df = pd.read_sql(query, connection)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_21844\2876990811.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [ ]:
print('행 수')
print(len(df))

print('컬럼별 결측치 수')
print(df.isnull().sum()) 

print(df.head(20))